## APPLICAZIONE SUL TEST SET DELLE MODIFICHE FATTE SUL TRAINING SET

In [ ]:
import pandas as pd
from constants import*
from constants import numeric_columns as num_cols
import numpy as np
import ast

test_df = pd.read_csv("test.csv")
#test_df = pd.read_csv("train.csv")

### Trasformazione \N di genres in NaN

In [2]:
#trasforma valori \N di genres in nan
test_df.loc[test_df["genres"] == r'\N', "genres"] = np.nan

### Trasformazione dei generi in liste di stringhe; per i record NaN vengono trasformati in [""]

In [3]:
# Replace NaN values with an empty string
test_df["genres"] = test_df["genres"].fillna("")

# Split genres into lists
test_df["genres"] = test_df["genres"].str.split(',')

# Debug: Check the data types within the lists
print(test_df['genres'].apply(lambda x: [type(genre) for genre in (x if isinstance(x, list) else [])]))

# Preview the updated genres column
print(test_df["genres"].head())

0                      [<class 'str'>, <class 'str'>]
1                                     [<class 'str'>]
2                                     [<class 'str'>]
3                                     [<class 'str'>]
4       [<class 'str'>, <class 'str'>, <class 'str'>]
                            ...                      
5473                                  [<class 'str'>]
5474    [<class 'str'>, <class 'str'>, <class 'str'>]
5475                                  [<class 'str'>]
5476                                  [<class 'str'>]
5477                                  [<class 'str'>]
Name: genres, Length: 5478, dtype: object
0             [Music, Short]
1                   [Horror]
2               [Reality-TV]
3                [Game-Show]
4    [Crime, Drama, Mystery]
Name: genres, dtype: object


### Trasformare i valori \N in nan nelle colonne numeriche

In [4]:
#uniformo assegnando NaN a tutti i valori nulli
for col in num_cols:
    #Converts the values in column col to numeric types (e.g., int, float). 
    # #If a value cannot be converted  (e.g., invalid strings), it is replaced with NaN (missing value).
    test_df[col] = pd.to_numeric(test_df[col], errors='coerce')

    #abbiamo trasformato i valori numerici in interi e non float (con Int64 accetta i Nan)
    #test_df[col] = test_df[col].astype('Int64')

### Ceiling per i rating

In [5]:
#ceiling per rating
test_df['rating'] = test_df['rating'].str.extract(
        r",\s*(\d+)\]"
    ).astype(int)

### CountryofOrigin come lista di stringhe

In [6]:
#traformazione paesi in liste di stringhe
test_df['countryOfOrigin'] = test_df['countryOfOrigin'].apply(ast.literal_eval)

print(test_df['countryOfOrigin'].apply(lambda x: [type(country) for country in x]))

0       [<class 'str'>]
1       [<class 'str'>]
2       [<class 'str'>]
3       [<class 'str'>]
4       [<class 'str'>]
             ...       
5473    [<class 'str'>]
5474    [<class 'str'>]
5475    [<class 'str'>]
5476    [<class 'str'>]
5477    [<class 'str'>]
Name: countryOfOrigin, Length: 5478, dtype: object


### GESTIONE 1 MISSING VALUES --> trasformazione awardWins a 0 per i nan

In [7]:
test_df["awardWins"] = test_df["awardWins"].fillna(0)

### GESTIONE 2 MISSING VALUES --> one-hot per generi; i missing hanno 0 in tutte le variabili

In [8]:
# Flatten all lists and get unique genres
all_genres = set(genre for sublist in test_df['genres'] if isinstance(sublist, list) for genre in sublist)
print(all_genres)

{'', 'Western', 'Thriller', 'Horror', 'Animation', 'Family', 'News', 'Sport', 'Reality-TV', 'Biography', 'Sci-Fi', 'Fantasy', 'Documentary', 'Talk-Show', 'Drama', 'Music', 'Game-Show', 'Adventure', 'Romance', 'Mystery', 'Short', 'Action', 'Adult', 'History', 'Musical', 'Crime', 'Film-Noir', 'War', 'Comedy'}


In [9]:
# Create a one-hot encoded column for each genre
for genre in all_genres:
    test_df[f"is_{genre}"] = test_df['genres'].apply(lambda x: 1 if genre in x else 0)

In [10]:
#droppiamo colonna ["is_"] così che il vettore per i record che non hanno genere siano tutti 0
test_df.drop(["is_"], axis=1, inplace=True)

### GESTIONE 3 MISSING VALUES --> sostituire i nan di runtimeMins come mediana raggruppato per titletype

In [11]:
#creiamo nuova colonna runtimeMinutes dove sostituiamo i nan con mediana 
test_df["fill_runtimeMinutes"] = test_df.groupby(["titleType"])["runtimeMinutes"].transform(lambda x: x.fillna(x.median()))

### Drop colonne redundant

In [13]:
test_df.drop(["bestRating"], axis=1, inplace=True)
test_df.drop(["worstRating"], axis=1, inplace=True)
test_df.drop(["isRatable"], axis=1, inplace=True)
test_df.drop(["ratingCount"], axis=1, inplace=True)

### Gestione country of origin

In [ ]:
HISTORIC_CODES = {
    # Europa
    'XYU': {'name': 'Yugoslavia', 'continent': 'Europe'},
    'YUCS': {'name': 'Yugoslavia (Socialist Federal Republic)', 'continent': 'Europe'},
    'YUG': {'name': 'Yugoslavia', 'continent': 'Europe'},
    'SUHH': {'name': 'Soviet Union', 'continent': 'Europe'},
    'SUN': {'name': 'Soviet Union', 'continent': 'Europe'},
    'SU': {'name': 'Soviet Union', 'continent': 'Europe'},
    'USSR': {'name': 'Union of Soviet Socialist Republics', 'continent': 'Europe'},
    'DDDE': {'name': 'East Germany', 'continent': 'Europe'},
    'DDR': {'name': 'German Democratic Republic', 'continent': 'Europe'},
    'XWG': {'name': 'East Germany', 'continent': 'Europe'},
    'BRD': {'name': 'Federal Republic of Germany (West Germany)', 'continent': 'Europe'},
    'FRGG': {'name': 'West Germany', 'continent': 'Europe'},
    'CSXX': {'name': 'Czechoslovakia', 'continent': 'Europe'},
    'CSHH': {'name': 'Czechoslovakia', 'continent': 'Europe'},
    'CSK': {'name': 'Czechoslovakia', 'continent': 'Europe'},
    'TCH': {'name': 'Czechoslovakia', 'continent': 'Europe'},
    'XCZ': {'name': 'Czechoslovakia (1945-1992)', 'continent': 'Europe'},
    'SCG': {'name': 'Serbia and Montenegro', 'continent': 'Europe'},
    'SCGN': {'name': 'Serbia and Montenegro', 'continent': 'Europe'},
    'XKV': {'name': 'Kosovo', 'continent': 'Europe'},
    'XPL': {'name': 'Poland (1945-1989)', 'continent': 'Europe'},
    'XHU': {'name': 'Hungary (1945-1989)', 'continent': 'Europe'},
    'XEU': {'name': 'European Union', 'continent': 'Europe'},
    'EUE': {'name': 'European Union', 'continent': 'Europe'},
    'XCI': {'name': 'Channel Islands', 'continent': 'Europe'},
    'XES': {'name': 'Estonia (pre-1991)', 'continent': 'Europe'},
    'XLV': {'name': 'Latvia (pre-1991)', 'continent': 'Europe'},
    'XLT': {'name': 'Lithuania (pre-1991)', 'continent': 'Europe'},
    'XAZ': {'name': 'Azores', 'continent': 'Europe'},
    'XFR': {'name': 'France (Historical)', 'continent': 'Europe'},
    'FXX': {'name': 'Metropolitan France', 'continent': 'Europe'},
    'AHU': {'name': 'Austria-Hungary', 'continent': 'Europe'},
    'PRU': {'name': 'Prussia', 'continent': 'Europe'},
    'BAV': {'name': 'Bavaria', 'continent': 'Europe'},
    'KPS': {'name': 'Kingdom of the Two Sicilies', 'continent': 'Europe'},
    'VEN': {'name': 'Republic of Venice', 'continent': 'Europe'},
    'XOH': {'name': 'Ottoman Hungary', 'continent': 'Europe'},
    'XCR': {'name': 'Crimean Khanate', 'continent': 'Europe'},
    'MON': {'name': 'Montenegro (pre-independence)', 'continent': 'Europe'},
    'VA': {'name': 'Vatican City State', 'continent': 'Europe'},
    'SRB': {'name': 'Kingdom of Serbia', 'continent': 'Europe'},
    'PDM': {'name': 'Piedmont-Sardinia', 'continent': 'Europe'},
    'XPP': {'name': 'Papal States', 'continent': 'Europe'},
    'XMO': {'name': 'Monaco (Historical)', 'continent': 'Europe'},
    'XSM': {'name': 'San Marino (Historical)', 'continent': 'Europe'},
    'XLI': {'name': 'Liechtenstein (Historical)', 'continent': 'Europe'},
    
    # Asia
    'ANT': {'name': 'Netherlands Antilles', 'continent': 'North_America'},
    'ANHH': {'name': 'Netherlands Antilles', 'continent': 'North_America'},
    'XAN': {'name': 'Netherlands Antilles', 'continent': 'North_America'},
    'BUR': {'name': 'Burma (now Myanmar)', 'continent': 'Asia'},
    'TPE': {'name': 'Chinese Taipei (Taiwan)', 'continent': 'Asia'},
    'XIR': {'name': 'Persia (now Iran)', 'continent': 'Asia'},
    'TMP': {'name': 'East Timor (pre-independence)', 'continent': 'Asia'},
    'TLS': {'name': 'East Timor', 'continent': 'Asia'},
    'XTI': {'name': 'East Timor (Portuguese Timor)', 'continent': 'Asia'},
    'YMD': {'name': 'South Yemen (People\'s Democratic Republic)', 'continent': 'Asia'},
    'YMN': {'name': 'North Yemen (Yemen Arab Republic)', 'continent': 'Asia'},
    'XHK': {'name': 'Hong Kong (pre-1997)', 'continent': 'Asia'},
    'HKJ': {'name': 'Hashemite Kingdom of Jordan', 'continent': 'Asia'},
    'XNA': {'name': 'Neutral Zone (Saudi Arabia-Iraq)', 'continent': 'Asia'},
    'XEA': {'name': 'East Asia (historical region)', 'continent': 'Asia'},
    'XSE': {'name': 'Southeast Asia (historical region)', 'continent': 'Asia'},
    'XME': {'name': 'Middle East (region)', 'continent': 'Asia'},
    'XIO': {'name': 'British Indian Ocean Territory', 'continent': 'Asia'},
    'XSI': {'name': 'Sikkim (now part of India)', 'continent': 'Asia'},
    #'XEG': {'name': 'United Arab Republic (Egypt & Syria)', 'continent': 'Africa/Asia'},
    'XMB': {'name': 'Manchukuo (Japanese puppet state in Manchuria)', 'continent': 'Asia'},
    'KOR': {'name': 'Korea (pre-division)', 'continent': 'Asia'},
    'XSM': {'name': 'Siam (now Thailand)', 'continent': 'Asia'},
    'XCY': {'name': 'Ceylon (now Sri Lanka)', 'continent': 'Asia'},
    'SAA': {'name': 'Sarawak (pre-Malaysia)', 'continent': 'Asia'},
    'SBH': {'name': 'North Borneo (now Sabah)', 'continent': 'Asia'},
    'XPH': {'name': 'Philippines (American period)', 'continent': 'Asia'},
    'XIN': {'name': 'British India', 'continent': 'Asia'},
    'GBG': {'name': 'Great Burhan Regime (Afghanistan)', 'continent': 'Asia'},
    'XPL': {'name': 'Palestine (British Mandate)', 'continent': 'Asia'},
    'XFI': {'name': 'French Indochina', 'continent': 'Asia'},
    'XDP': {'name': 'Portuguese India', 'continent': 'Asia'},
    'XMO': {'name': 'Macao (pre-1999)', 'continent': 'Asia'},
    'XQG': {'name': 'Qing Dynasty China', 'continent': 'Asia'},
    
    # Africa
    'RHO': {'name': 'Rhodesia (now Zimbabwe)', 'continent': 'Africa'},
    'XRH': {'name': 'Southern Rhodesia', 'continent': 'Africa'},
    'ZAR': {'name': 'Zaire (now Democratic Republic of the Congo)', 'continent': 'Africa'},
    'XAF': {'name': 'Afars and Issas (now Djibouti)', 'continent': 'Africa'},
    'XDY': {'name': 'Dahomey (now Benin)', 'continent': 'Africa'},
    'XUG': {'name': 'Uganda (pre-1962)', 'continent': 'Africa'},
    'XET': {'name': 'Ethiopia (pre-1993)', 'continent': 'Africa'},
    'XSU': {'name': 'Sudan (pre-2011)', 'continent': 'Africa'},
    'USR': {'name': 'Upper Volta (now Burkina Faso)', 'continent': 'Africa'},
    'XVO': {'name': 'Upper Volta', 'continent': 'Africa'},
    'XBF': {'name': 'Burkina Faso (1984-present)', 'continent': 'Africa'},
    'XLI': {'name': 'Libya (pre-2011)', 'continent': 'Africa'},
    'XTC': {'name': 'Tanganyika (now part of Tanzania)', 'continent': 'Africa'},
    'XZR': {'name': 'Zanzibar (now part of Tanzania)', 'continent': 'Africa'},
    'XTZ': {'name': 'Tanzania (1964-present)', 'continent': 'Africa'},
    'XGH': {'name': 'Gold Coast (now Ghana)', 'continent': 'Africa'},
    'XSL': {'name': 'Sierra Leone (pre-1961)', 'continent': 'Africa'},
    'XMA': {'name': 'Madeira', 'continent': 'Africa'},
    'XTA': {'name': 'Tangier International Zone', 'continent': 'Africa'},
    'XCV': {'name': 'Cape Verde (before 2013)', 'continent': 'Africa'},
    'XBI': {'name': 'British Somaliland', 'continent': 'Africa'},
    'XIF': {'name': 'Italian Somaliland', 'continent': 'Africa'},
    'XAL': {'name': 'Algérie française (French Algeria)', 'continent': 'Africa'},
    'XNY': {'name': 'Nyasaland (now Malawi)', 'continent': 'Africa'},
    'XBE': {'name': 'Belgian Congo', 'continent': 'Africa'},
    'XBA': {'name': 'Basutoland (now Lesotho)', 'continent': 'Africa'},
    'XSW': {'name': 'South West Africa (now Namibia)', 'continent': 'Africa'},
    'XZB': {'name': 'Zambezi (colonial term)', 'continent': 'Africa'},
    'XAO': {'name': 'Angola (Portuguese colony)', 'continent': 'Africa'},
    'EH': {'name': 'Western Sahara (Sahara Occidental)', 'continent': 'Africa'},
    'XEN': {'name': 'Emirates of North Africa', 'continent': 'Africa'},
    'XTR': {'name': 'Tripolitania (now part of Libya)', 'continent': 'Africa'},
    'XCY': {'name': 'Cyrenaica (now part of Libya)', 'continent': 'Africa'},
    'XAB': {'name': 'Abyssinia (historical Ethiopia)', 'continent': 'Africa'},
    'XBB': {'name': 'Bechuanaland (now Botswana)', 'continent': 'Africa'},
    'XMA': {'name': 'Madagascar (French colony)', 'continent': 'Africa'},
    'XMO': {'name': 'Morocco (French/Spanish protectorate)', 'continent': 'Africa'},
    'XTU': {'name': 'Tunisia (French protectorate)', 'continent': 'Africa'},
    
    # Americas
    'XUS': {'name': 'United States (Historical)', 'continent': 'North_America'},
    'XCA': {'name': 'Canada (Historical)', 'continent': 'North_America'},
    'XPU': {'name': 'Panama Canal Zone', 'continent': 'North_America'},
    'XMX': {'name': 'Mexico (Historical)', 'continent': 'North_America'},
    'XBR': {'name': 'Brazil (Historical)', 'continent': 'South_America'},
    'XAR': {'name': 'Argentina (Historical)', 'continent': 'South_America'},
    'XCB': {'name': 'Caribbean Islands (collective)', 'continent': 'North_America'},
    'XCL': {'name': 'Chile (pre-1990)', 'continent': 'South_America'},
    'XCO': {'name': 'Colombia (pre-1991)', 'continent': 'South_America'},
    'XCU': {'name': 'Cuba (pre-1959)', 'continent': 'North_America'},
    'XEN': {'name': 'English Caribbean Islands', 'continent': 'North_America'},
    'XFR': {'name': 'French Caribbean Islands', 'continent': 'North_America'},
    'XGL': {'name': 'Greenland (pre-1979)', 'continent': 'North_America'},
    'XGY': {'name': 'Guyana (British Guiana)', 'continent': 'South_America'},
    'XSR': {'name': 'Suriname (Dutch Guiana)', 'continent': 'South_America'},
    'XGF': {'name': 'French Guiana (Historical)', 'continent': 'South_America'},
    'XPI': {'name': 'Saint-Pierre and Miquelon', 'continent': 'North_America'},
    'USC': {'name': 'Confederate States of America', 'continent': 'North_America'},
    'XTX': {'name': 'Republic of Texas', 'continent': 'North_America'},
    'XHT': {'name': 'Saint-Domingue (now Haiti)', 'continent': 'North_America'},
    'XBH': {'name': 'British Honduras (now Belize)', 'continent': 'North_America'},
    'XBW': {'name': 'British West Indies', 'continent': 'North_America'},
    'XUN': {'name': 'United Provinces of Central America', 'continent': 'North_America'},
    'XGP': {'name': 'Gran Colombia', 'continent': 'South_America'},
    'XPR': {'name': 'Peru-Bolivian Confederation', 'continent': 'South_America'},
    'XPN': {'name': 'Panama (as part of Colombia)', 'continent': 'North_America'},
    'XRI': {'name': 'Río de la Plata (Viceroyalty)', 'continent': 'South_America'},
    'XLN': {'name': 'New Granada (Viceroyalty)', 'continent': 'South_America'},
    'XNS': {'name': 'New Spain (Viceroyalty)', 'continent': 'North_America'},
    'XBE': {'name': 'British Empire in America', 'continent': 'North_America'},
    'XDW': {'name': 'Danish West Indies (now US Virgin Islands)', 'continent': 'North_America'},
    
    # Oceania
    'XAU': {'name': 'Australia (pre-Federation)', 'continent': 'Oceania'},
    'XNZ': {'name': 'New Zealand (pre-1907)', 'continent': 'Oceania'},
    'PCI': {'name': 'Pacific Islands Trust Territory', 'continent': 'Oceania'},
    'XPI': {'name': 'Pacific Islands (US Trust Territory)', 'continent': 'Oceania'},
    'XFJ': {'name': 'Fiji (pre-1970)', 'continent': 'Oceania'},
    'XPN': {'name': 'Papua New Guinea (pre-1975)', 'continent': 'Oceania'},
    'XPW': {'name': 'Palau (pre-1994)', 'continent': 'Oceania'},
    'XMH': {'name': 'Marshall Islands (pre-1986)', 'continent': 'Oceania'},
    'XFM': {'name': 'Federated States of Micronesia (pre-1986)', 'continent': 'Oceania'},
    'XGE': {'name': 'Gilbert Islands (now part of Kiribati)', 'continent': 'Oceania'},
    'XEL': {'name': 'Ellice Islands (now Tuvalu)', 'continent': 'Oceania'},
    'XNH': {'name': 'New Hebrides (now Vanuatu)', 'continent': 'Oceania'},
    'PCT': {'name': 'Pacific Islands Trust Territory', 'continent': 'Oceania'},
    'XWS': {'name': 'Western Samoa (now Samoa)', 'continent': 'Oceania'},
    'XNA': {'name': 'Netherlands New Guinea', 'continent': 'Oceania'},
    'XHS': {'name': 'Hawaii (pre-US statehood)', 'continent': 'Oceania'},
    'XBP': {'name': 'British Polynesia', 'continent': 'Oceania'},
    'XFP': {'name': 'French Polynesia (historical)', 'continent': 'Oceania'},
    'XNC': {'name': 'New Caledonia (historical)', 'continent': 'Oceania'},
    'XSI': {'name': 'Solomon Islands (British protectorate)', 'continent': 'Oceania'},
    
}

In [ ]:
import pycountry
import pycountry_convert as pc


def get_continent(country_code):
    """Ottiene il continente per una singola sigla di paese"""
    # Controlla prima nel dizionario storico
    if country_code in HISTORIC_CODES:
        return HISTORIC_CODES[country_code]['continent']
    
    try:
        # Per codici a 2 lettere (alpha-2)
        if len(country_code) == 2:
            country_continent_code = pc.country_alpha2_to_continent_code(country_code)
            continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
            return continent_name
            
        # Per codici a 3 lettere (alpha-3)
        elif len(country_code) == 3:
            # Converti da alpha-3 a alpha-2
            country = pycountry.countries.get(alpha_3=country_code)
            if country:
                country_alpha2 = country.alpha_2
                country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
                continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
                return continent_name
            
        # Controlla nei paesi storici di pycountry
        historic = next((c for c in pycountry.historic_countries if c.alpha_3 == country_code), None)
        if historic:
            return map_historic_to_continent(historic.name)
                
        return None
    except (KeyError, ValueError, AttributeError):
        return None

def get_continents_for_list(country_codes):
    """Gestisce una lista di sigle di paesi e restituisce un dizionario con i risultati"""
    results = {}
    
    # Se l'input è una singola stringa, convertila in lista
    if isinstance(country_codes, str):
        country_codes = [country_codes]
    
    # Elabora ogni sigla nella lista
    for code in country_codes:
        results[code] = get_continent(code)
    
    return results

def map_historic_to_continent(country_name):
    """Mappa manuale di alcuni paesi storici ai loro continenti"""
    mappings = {
        'USSR': 'Europe',
        'YUGOSLAVIA': 'Europe',
        'CZECHOSLOVAKIA': 'Europe',
        # Aggiungi altri paesi storici secondo necessità
    }
    
    for key, value in mappings.items():
        if key in country_name.upper():
            return value
    
    return None

# utilizzo con lista
all_countries = set(country for sublist in test_df['countryOfOrigin'] if isinstance(sublist, list) for country in sublist)
continent_mapping = get_continents_for_list(all_countries)
continent_mapping

In [ ]:
def replace_country_with_continent(df, column_name, continent_mapping):
    """
    Sostituisce i codici dei paesi nella colonna di un DataFrame con il loro continente corrispondente.
    
    :param df: DataFrame contenente la colonna da modificare
    :param column_name: Nome della colonna con le liste di codici paese
    :param continent_mapping: Dizionario {codice_paese: continente}
    :return: DataFrame con la colonna modificata
    """
    df[column_name] = df[column_name].apply(
        lambda country_list: list(set(continent_mapping.get(country, 'Unknown') for country in country_list))
    )
    return df


# Applicazione della funzione
df = replace_country_with_continent(test_df, 'countryOfOrigin', get_continents_for_list(all_countries))

# Stampa del risultato
print(test_df["countryOfOrigin"])


crea attributi is_from_X ->onehotencoding perchè ci sono multivalori

In [ ]:
# Flatten all lists and get unique genres
all_continents = set(cont for sublist in test_df['countryOfOrigin'] if isinstance(sublist, list) for cont in sublist)
print(all_continents)

# Create a one-hot encoded column for each continent
for genre in all_continents:
    test_df[f"is_from_{genre}"] = test_df['countryOfOrigin'].apply(lambda x: 1 if genre in x else 0)


## MANCANO -> nel pp_train usato in alcuni clustering non ci sono:
- Totalmedia -> images + video (usato nel hierachical)
- total nomination -> award win + award exclude (usato nel hierachical)
- End year -> rimpiazza con start year, ma non per le serie? restano nan
- fill runtime -> sarebbe da cambiare con 30-70 -> nel clustering l'abbiamo usato con mediana -> ne vale la pena?
- can have episode convertire in 0/1? per knn e nb si, meglio averle coerenti e non qualcuna 0/1 e qualcuna True/False --> pp_class['canHaveEpisodes'] = pp_class['canHaveEpisodes'].astype(int)

### Applicazione delle modifiche al nuovo csv

importante che mentre fa uno l'altro sia commentato

In [ ]:
 #TEST
test_df.to_csv('pp_test.csv', index=False, header=True)
#TRAIN PP_TRAIN_COMPLETO ->
#test_df.to_csv("pp_train_completo.csv", index=False, header=True)